In [35]:
import numpy as np

In [36]:
## Constants

service_times = {
    "contract"  :   30,
    "complain"  :   25,
    "confirm"   :   10,
    "request"   :   5, 
    "review"    :   10,
}

transition_matrices = {
    "A" : {
        "services": ["complain", "contract"], 
        "matrix" : [
            [0.9, 0.1], [0.2, 0.8],
        ],
        "time_between": 5
    },
    "B" : {
        "services": ["review", "request"], 
        "matrix" : [
            [0.95, 0.05], [0.15, 0.85],
        ],
        "time_between": 7
    },
    "C" : {
        "services": ["review", "request", "confirm"], 
        "matrix" : [
            [0.75, 0.15, 0.1], [0.1, 0.8, 0.1], [0.05, 0.05, 0.9],
        ],
        "time_between": 10
    },
}

interarrival_times = {
    "contract"  :   ["minute", "normal", 40, 36],
    "complain"  :   ["hour", "exponential", 0.5],
    "confirm"   :   ["hour", "gamma", 1, 2],
    "request"   :   ["minute", "exponential", 0.06], 
    "review"    :   ["minute", "normal", 15, 36],
}


In [33]:
class ServiceType:
    def __init__(self, name):
        self.name = name
        self.service_time = service_times[name]

    def get_service_time(self):
        scale = 1.0 / self.service_time
        sample = np.random.exponential(scale=scale)
        return sample 
    
    def get_interarrival_time(self): 
        config = interarrival_times[self.name]
        scale, distr, params = config[0], config[1], config[2:]
        sample = 0
        if distr == "normal": 
            sample = np.random.normal(loc=params[0], scale=params[1]**0.5)
        elif distr == "exponential": 
            sample = np.random.exponential(scale=1/params[0])
        elif distr == "gamma": 
            sample = np.random.gamma(scale=params[0], shape=params[1])
        
        if scale == "hour": 
            sample *= 3600
        elif scale == "minute": 
            sample *= 60 
        
        return np.math.ceil(sample)        

class WorkerType: 
    def __init__(self, name):
        config = transition_matrices[name]
        self.trans_dict = {}
        services = config["services"]
        for i in range(len(services)) :
            self.trans_dict[services[i]] = config["matrix"][i]
        self.transition_time = config["time_between"]
    
    def get_first_job(self): 
        job_name = np.random.choice(self.trans_dict.keys())
        return ServiceType(job_name)
          
class Worker: 
    def __init__(self, type: WorkerType):
        self.type = type
        self.current_job = type.get_first_job()

class Customer: 
    def __init__(self, type: ServiceType, enter_time): 
        self.type = type 
        self.enter_time = enter_time
        self.service_time = type.get_service_time()


class Service: 
    def __init__(self, type: ServiceType):
        self.type = type 
        self.queue = []
        self.customers = []
        self.next_customer = self.type.get_interarrival_time()

        

In [26]:
def simulate(workers_count, simulation_time):

    services = []
    workers = []

    for service, time in service_times.items(): 
        service_type = ServiceType(service, time)
        service = Service(service_type)
        services.append(service)

    for job in transition_matrices: 
        worker_type = WorkerType(job)
        for i in range(workers_count): 
            w = Worker(worker_type)



    # for i in range(simulation_time): # each iteratnio is 1 minute



In [27]:
simulate(2, 10)

contract 30
complain 25
confirm 10
request 5
review 10
